In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from imblearn import metrics as metrics2
from sklearn import model_selection
from gensim.corpora import Dictionary
import pickle
import urllib.request

In [2]:
''' load file bow_corpus '''

url = 'https://raw.githubusercontent.com/namvnvx/DS107_Project/main/Corpus/bow_corpus.pkl'
with urllib.request.urlopen(url) as f:
    bow_corpus = pickle.load(f)

In [3]:
''' load dữ liệu train đã vector hóa và phân cụm '''

result = pd.read_csv('https://raw.githubusercontent.com/namvnvx/DS107_Project/main/Demo_data/Vectorized_Clustered_TrainData.csv')

In [8]:
score = {}
model = RandomForestClassifier()

for i in range(0,5,1):
  data_cluster_X = result.loc[result['cluster'] == i].drop(columns = ['class', 'cluster'])
  data_cluster_y = result['class'].loc[result['cluster'] == i]

  y_pred = model_selection.cross_val_predict(model, data_cluster_X, data_cluster_y, cv = 10)

  Accuracy = metrics.accuracy_score(data_cluster_y, y_pred)
  F_measure = metrics.f1_score(data_cluster_y, y_pred, average = 'weighted')
  Geometric_Mean = metrics2.geometric_mean_score(data_cluster_y, y_pred, average = 'weighted')
  Sensitivity = metrics2.sensitivity_score(data_cluster_y, y_pred, average = 'weighted')
  Specificity = metrics2.specificity_score(data_cluster_y, y_pred, average = 'weighted')

  avg = (Accuracy + F_measure + Geometric_Mean + Sensitivity + Specificity)
  score[i] = avg

In [9]:
score

{0: 2.9237653170539524,
 1: 2.664335014102949,
 2: 2.4886412986333597,
 3: 2.5578965573461776,
 4: 2.543435066050151}

In [19]:
#Tỉ lệ phân cụm
lenght = len(result['cluster'])
for i in result['cluster'].unique():
  count = len(result.loc[result['cluster'] == i])
  rate = (count / lenght)*100
  print(f'\n_{i}_ : {count}/{lenght} row --- {rate:.3f} %' )


_0_ : 46200/114735 row --- 40.267 %

_2_ : 24108/114735 row --- 21.012 %

_4_ : 14711/114735 row --- 12.822 %

_1_ : 15001/114735 row --- 13.074 %

_3_ : 14715/114735 row --- 12.825 %


In [10]:
idscore = 0
max = score[idscore]
for key in score.keys():
  if score[key] > max:
    max = score[key]
    idscore = key
print(f'Cluster {idscore} has highest score')

Cluster 0 has highest score


In [11]:
  # negative
d1 = result.loc[result['cluster'] == idscore].loc[result['class'] == 0].drop(columns = ['class','cluster'])
data_neg = d1.drop(columns=['Unnamed: 0'])
  # neutral
d2= result.loc[result['cluster'] == idscore].loc[result['class'] == 1].drop(columns = ['class','cluster'])
data_neu = d2.drop(columns=['Unnamed: 0'])
  # positive
d3 = result.loc[result['cluster'] == idscore].loc[result['class'] == 2].drop(columns = ['class','cluster'])
data_pos = d3.drop(columns=['Unnamed: 0'])

In [12]:
data_neg

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,7,5,3,2,4,0,1,-1,6,0,0,0,0,0,0,0,0,0,0
1,10,12,9,-1,8,14,13,11,0,0,0,0,0,0,0,0,0,0,0
8,13,46,44,-1,43,17,45,47,0,0,0,0,0,0,0,0,0,0,0
10,51,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,90,87,92,85,88,92,-1,94,93,86,89,91,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114693,354,1147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
114703,67,242,1066,1445,280,-1,0,0,0,0,0,0,0,0,0,0,0,0,0
114709,182,-1,430,787,786,788,1590,103,0,0,0,0,0,0,0,0,0,0,0
114719,495,762,284,181,114,767,154,278,1865,0,0,0,0,0,0,0,0,0,0


In [13]:
data_neu

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
2,16,15,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,25,-1,11,28,29,27,26,0,0,0,0,0,0,0,0,0,0,0,0
6,39,37,-1,36,-1,-1,11,38,0,0,0,0,0,0,0,0,0,0,0
7,42,41,-1,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,49,48,50,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114722,790,1165,801,11,1302,377,-1,-1,0,0,0,0,0,0,0,0,0,0,0
114726,-1,103,261,3173,27,621,0,0,0,0,0,0,0,0,0,0,0,0,0
114727,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
114729,533,986,981,49,16,911,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
data_pos

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
3,19,17,22,23,21,20,18,-1,24,0,0,0,0,0,0,0,0,0,0
5,30,-1,34,-1,-1,31,33,32,35,0,0,0,0,0,0,0,0,0,0
11,57,56,54,58,59,55,53,-1,52,0,0,0,0,0,0,0,0,0,0
14,76,73,74,77,75,-1,0,0,0,0,0,0,0,0,0,0,0,0,0
16,-1,82,83,-1,84,-1,81,-1,80,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114679,962,1592,16,257,16,1153,0,0,0,0,0,0,0,0,0,0,0,0,0
114692,994,103,103,-1,106,1650,116,911,0,0,0,0,0,0,0,0,0,0,0
114711,1038,460,1260,125,271,0,0,0,0,0,0,0,0,0,0,0,0,0,0
114715,182,197,1795,100,1151,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
def bag_of_word(data_neu):
  bow_sentiment = []
  # dữ liệu trong data_neu
  data_neu_filtered = data_neu.apply(lambda row: [x for x in row if x != 0], axis=1)
  # tập set các corpus
  bow_corpus_set = [set([t[0] for t in indice]) for indice in bow_corpus]

  bow_corpus_set_filtered = []
  for bow_element in bow_corpus_set:
      filtered_set = {element for element in bow_element if element != 0}
      bow_corpus_set_filtered.append(filtered_set)

  for row in data_neu_filtered:
    for index, bow_ele in enumerate(bow_corpus_set_filtered):
      if all(element in bow_ele for element in row):
        bow_sentiment.append(bow_corpus[index])
        break
  return bow_sentiment

In [16]:
bow_neu = bag_of_word(data_neu)
bow_pos = bag_of_word(data_pos)
bow_neg = bag_of_word(data_neg)

In [17]:
pickle.dump(bow_neu, open('neutral_corpus.pkl', 'wb'))
pickle.dump(bow_pos, open('positive_corpus.pkl', 'wb'))
pickle.dump(bow_neg, open('negative_corpus.pkl', 'wb'))